## <center> JantaHack : Recommendation Systems</center>
_______________

<center><img src="https://datahack-prod.s3.ap-south-1.amazonaws.com/__sized__/contest_cover/cover_81npx51-thumbnail-1200x1200.png"/></center>

### Problem Statement

> - Your client is a fast-growing mobile platform, for hosting coding challenges. They have a unique business model, where they crowdsource problems from various creators(authors). These authors create the problem and release it on the client's platform. The users then select the challenges they want to solve. The authors make money based on the level of difficulty of their problems and how many users take up their challenge.

 
> 
> - The client, on the other hand makes money when the users can find challenges of their interest and continue to stay on the platform. Till date, the client has relied on its domain expertise, user interface and experience with user behaviour to suggest the problems a user might be interested in. You have now been appointed as the data scientist who needs to come up with the algorithm to keep the users engaged on the platform.

## What to recommend

> - The client has provided you with history of last 10 challenges the user has solved, and you need to predict which might be the next 3 challenges the user might be interested to solve. Apply your data science skills to help the client make a big mark in their user engagements/revenue.

## Data Description
for data desciption, there are three files:

- ```train.csv```: It contains the set of 13 challenges that were attempted by the same user in a sequence.
- ```challenge_data.csv```: Contains attributes related to each challenge
- ```test.csv```: Contains the first 10 challenges solved by a new user set (not in train) in the test set. We need to predict


![](https://raw.githubusercontent.com/AIVenture0/JanataHack-Recommendation-Systems/master/data_description1.jpg)

![](https://raw.githubusercontent.com/AIVenture0/JanataHack-Recommendation-Systems/master/data_description2.jpg)

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/av-recommendation-systems/sample_submission_J0OjXLi_DDt3uQN.csv
/kaggle/input/av-recommendation-systems/train_mddNHeX/challenge_data.csv
/kaggle/input/av-recommendation-systems/train_mddNHeX/train.csv
/kaggle/input/av-recommendation-systems/test_HLxMpl7/test.csv


In [2]:
# To print multiple output in a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [3]:
os.getcwd()

'/kaggle/working'

In [4]:
# Load Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Dense,Embedding,LSTM,Dropout,Bidirectional,Input,BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping,ModelCheckpoint


Using TensorFlow backend.


## Reading the train and test data

In [5]:
train = pd.read_csv("../input/av-recommendation-systems/train_mddNHeX/train.csv")
test = pd.read_csv("../input/av-recommendation-systems/test_HLxMpl7/test.csv")

train.head()
test.head()


,user_sequence,user_id,challenge_sequence,challenge
0,4576_1,4576,1,CI23714
1,4576_2,4576,2,CI23855
2,4576_3,4576,3,CI24917
3,4576_4,4576,4,CI23663
4,4576_5,4576,5,CI23933


,user_sequence,user_id,challenge_sequence,challenge
0,4577_1,4577,1,CI23855
1,4577_2,4577,2,CI23933
2,4577_3,4577,3,CI24917
3,4577_4,4577,4,CI24915
4,4577_5,4577,5,CI23714


In [6]:
train[train.challenge_sequence > 10]

,user_sequence,user_id,challenge_sequence,challenge
10,4576_11,4576,11,CI24958
11,4576_12,4576,12,CI23667
12,4576_13,4576,13,CI23691
23,4580_11,4580,11,CI24915
24,4580_12,4580,12,CI25727
...,...,...,...,...
903901,113837_12,113837,12,CI23990
903902,113837_13,113837,13,CI25674
903913,113839_11,113839,11,CI28610
903914,113839_12,113839,12,CI26388


In [7]:
# Create labels
label = train[train.challenge_sequence > 10][['user_id','challenge']]
label.rename(columns={'challenge':'label'},inplace=True)

In [8]:
label.head()

,user_id,label
10,4576,CI24958
11,4576,CI23667
12,4576,CI23691
23,4580,CI24915
24,4580,CI25727


In [9]:
# Treat the sequence of challenges as text
df = train[train.challenge_sequence <= 10].groupby('user_id').challenge.aggregate(lambda x: ' '.join(x)).reset_index()
df.head()

,user_id,challenge
0,4576,CI23714 CI23855 CI24917 CI23663 CI23933 CI2513...
1,4580,CI23663 CI23855 CI23933 CI23975 CI24530 CI2371...
2,4581,CI26155 CI26156 CI26157 CI26158 CI26159 CI2616...
3,4582,CI23855 CI24915 CI24917 CI23933 CI23663 CI2495...
4,4585,CI23855 CI23975 CI24917 CI25135 CI23848 CI2371...


In [10]:
# Merge Labels
df = df.merge(label)
df.head()
df.shape

,user_id,challenge,label
0,4576,CI23714 CI23855 CI24917 CI23663 CI23933 CI2513...,CI24958
1,4576,CI23714 CI23855 CI24917 CI23663 CI23933 CI2513...,CI23667
2,4576,CI23714 CI23855 CI24917 CI23663 CI23933 CI2513...,CI23691
3,4580,CI23663 CI23855 CI23933 CI23975 CI24530 CI2371...,CI24915
4,4580,CI23663 CI23855 CI23933 CI23975 CI24530 CI2371...,CI25727


(208596, 3)

In [11]:

# Validation split for early stopping
df_train, df_validation = train_test_split(df.sample(frac=1,random_state=123), test_size=0.05, random_state=123)


In [12]:
df_train.head()
df_validation.head()

df_train.shape
df_validation.shape

,user_id,challenge,label
43773,27556,CI23836 CI24024 CI24958 CI23791 CI24157 CI2418...,CI26755
80313,46677,CI23887 CI23933 CI24138 CI24871 CI23769 CI2495...,CI24030
38680,24951,CI26938 CI27040 CI27044 CI27045 CI27047 CI2704...,CI26177
142050,79110,CI26894 CI26898 CI26899 CI26901 CI26900 CI2689...,CI26221
106266,60263,CI27028 CI27035 CI27041 CI27043 CI27044 CI2704...,CI27050


,user_id,challenge,label
25103,17885,CI23662 CI23714 CI24532 CI23691 CI25075 CI2385...,CI24958
135568,75737,CI27039 CI27046 CI27048 CI27050 CI27051 CI2704...,CI26198
180487,99199,CI25070 CI25079 CI25098 CI23975 CI24917 CI2385...,CI23600
177614,97723,CI26858 CI26863 CI26872 CI26877 CI26875 CI2687...,CI26220
118573,66832,CI23913 CI25024 CI25032 CI27003 CI27002 CI2700...,CI27001


(198166, 3)

(10430, 3)

In [13]:
# Load all the challenges
challenges = pd.read_csv('../input/av-recommendation-systems/train_mddNHeX/challenge_data.csv')

challenges.head()

,challenge_ID,programming_language,challenge_series_ID,total_submissions,publish_date,author_ID,author_gender,author_org_ID,category_id
0,CI23478,2,SI2445,37.0,06-05-2006,AI563576,M,AOI100001,NaN
1,CI23479,2,SI2435,48.0,17-10-2002,AI563577,M,AOI100002,32.0
2,CI23480,1,SI2435,15.0,16-10-2002,AI563578,M,AOI100003,NaN
3,CI23481,1,SI2710,236.0,19-09-2003,AI563579,M,AOI100004,70.0
4,CI23482,2,SI2440,137.0,21-03-2002,AI563580,M,AOI100005,NaN


In [14]:
len(challenges.challenge_ID.unique())

5606

In [15]:
len(df_train.label.unique())

4490

In [16]:
df_train.head()

,user_id,challenge,label
43773,27556,CI23836 CI24024 CI24958 CI23791 CI24157 CI2418...,CI26755
80313,46677,CI23887 CI23933 CI24138 CI24871 CI23769 CI2495...,CI24030
38680,24951,CI26938 CI27040 CI27044 CI27045 CI27047 CI2704...,CI26177
142050,79110,CI26894 CI26898 CI26899 CI26901 CI26900 CI2689...,CI26221
106266,60263,CI27028 CI27035 CI27041 CI27043 CI27044 CI2704...,CI27050


In [17]:
# Encode challenges
encoder = LabelEncoder()
encoder.fit(challenges['challenge_ID'])

df_train['brand_id_encoded'] = encoder.transform(df_train.label)
df_validation['brand_id_encoded'] = encoder.transform(df_validation.label)

# Tokenize text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_train['challenge'])


LabelEncoder()

In [18]:
len(tokenizer.word_index)

4961

In [29]:
# Constants
NB_WORDS = len(tokenizer.word_index)+1
MAX_SEQUENCE_LENGTH = 10
N_CATEGORIES = challenges.shape[0]

# Create sequences
sequences_train = tokenizer.texts_to_sequences(df_train['challenge'])
sequences_validation = tokenizer.texts_to_sequences(df_validation['challenge'])

# Pad sequences
x_train = pad_sequences(sequences_train, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')
x_validation = pad_sequences(sequences_validation, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')

# Set Labels
y_train = df_train['brand_id_encoded'].values
y_validation= df_validation['brand_id_encoded'].values

In [30]:
y_train
y_validation

array([3277,  552, 2699, ..., 3565, 3450, 1212])

array([1480, 2720,  122, ...,  752, 3715,  959])

In [31]:
# NN architecture
def get_model(path='',lr=0.001):
    adam = Adam(lr=lr)
    inp = Input(shape=(MAX_SEQUENCE_LENGTH, ))
    x = Embedding(NB_WORDS,256)(inp)
    x = BatchNormalization()(x)
    x = Bidirectional(LSTM(256, dropout=0.1, recurrent_dropout=0.1))(x)
    x = Dropout(0.4)(x)
    x = Dense(N_CATEGORIES, activation="softmax")(x)
    model = Model(inputs=inp, outputs=x)
    if path != '':
        model.load_weights(path)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

In [32]:
# Initialize the model
model = get_model()

In [33]:
# Model callbacks
path = 'best_model_weights'
es_callback = EarlyStopping(monitor='val_loss')
mc_callback = ModelCheckpoint('{}.hdf5'.format(path), monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True, mode='auto', period=1)
callbacks = [es_callback,mc_callback]

In [34]:
# Fit the model
model.fit(x_train,
          y_train,
          epochs=100,
          batch_size=500,
          validation_data=(x_validation, y_validation),
#           callbacks = callbacks
         )

Epoch 1/100
397/397 [==============================] - 45s 113ms/step - loss: 5.5485 - accuracy: 0.0458 - val_loss: 5.0095 - val_accuracy: 0.0581
Epoch 2/100
397/397 [==============================] - 43s 108ms/step - loss: 4.6258 - accuracy: 0.0783 - val_loss: 4.5238 - val_accuracy: 0.0782
Epoch 3/100
397/397 [==============================] - 44s 112ms/step - loss: 4.2513 - accuracy: 0.1019 - val_loss: 4.4062 - val_accuracy: 0.0848
Epoch 4/100
397/397 [==============================] - 45s 112ms/step - loss: 3.9825 - accuracy: 0.1186 - val_loss: 4.3617 - val_accuracy: 0.0820
Epoch 5/100
397/397 [==============================] - 44s 110ms/step - loss: 3.7787 - accuracy: 0.1326 - val_loss: 4.3487 - val_accuracy: 0.0797
Epoch 6/100
397/397 [==============================] - 43s 108ms/step - loss: 3.6282 - accuracy: 0.1441 - val_loss: 4.3653 - val_accuracy: 0.0780
Epoch 7/100
397/397 [==============================] - 45s 112ms/step - loss: 3.5073 - accuracy: 0.1543 - val_loss: 4.4058 -

Epoch 57/100
397/397 [==============================] - 44s 112ms/step - loss: 2.5227 - accuracy: 0.2422 - val_loss: 5.3511 - val_accuracy: 0.0156
Epoch 58/100
397/397 [==============================] - 43s 109ms/step - loss: 2.5227 - accuracy: 0.2410 - val_loss: 5.3672 - val_accuracy: 0.0181
Epoch 59/100
397/397 [==============================] - 44s 112ms/step - loss: 2.5168 - accuracy: 0.2410 - val_loss: 5.3783 - val_accuracy: 0.0184
Epoch 60/100
397/397 [==============================] - 44s 111ms/step - loss: 2.5129 - accuracy: 0.2423 - val_loss: 5.3773 - val_accuracy: 0.0146
Epoch 61/100
397/397 [==============================] - 44s 111ms/step - loss: 2.5041 - accuracy: 0.2419 - val_loss: 5.3992 - val_accuracy: 0.0157
Epoch 62/100
397/397 [==============================] - 43s 109ms/step - loss: 2.4994 - accuracy: 0.2432 - val_loss: 5.4140 - val_accuracy: 0.0176
Epoch 63/100
397/397 [==============================] - 44s 111ms/step - loss: 2.4972 - accuracy: 0.2435 - val_loss: 5

In [35]:
# Load best weights
# model = get_model('{}.hdf5'.format(path))

model=model

In [ ]:
# Test preprocessing
def padding(text):
    return pad_sequences(tokenizer.texts_to_sequences(text), maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')
test_text = test[test.challenge_sequence <= 10].groupby('user_id').challenge.aggregate(lambda x: ' '.join(x)).reset_index()
x_test = padding(test_text.challenge)

# Get top 3 predictions for each user
pred = model.predict(x_test,batch_size=2048)
pred = pred.argsort(axis=1)[:,-3:][:,::-1]

In [28]:
# Write Predictions
df_list = []
for i in range(3):
    test_11 = test_text[['user_id']]
    test_11['user_sequence'] = test_11.user_id.astype(str) + '_'+str(i+11)
    test_11['challenge'] = encoder.inverse_transform(pred[:,i])
    df_list.append(test_11[['user_sequence','challenge']])
pd.concat(df_list).to_csv('bes_submission8.csv',index=False)
